# Завдання на використання SQL в Python

1.Встановіть бібліотеку `mysql-connector-python` через pip aбо через conda.

In [5]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 2.4 MB/s eta 0:00:0000:0100:01m


In [7]:
import mysql.connector
from mysql.connector.connection import MySQLConnection
import datetime

2.В змінній `con` створіть підключення до бази даних employees за аналогією того, як це робилось в лекції:

In [9]:
con = mysql.connector.connect(user='user1', password='7oygWy6P',
                              host='3.68.62.36',
                              database='employees')

3.Виберіть максимальну і мінімальну дати найму співробітників використовуючи `con.cursor()`. Аби отримати результати в кінці, виконайте `cursor.fetchall()`.

In [17]:
cursor = con.cursor()
sql = """
SELECT MIN(hire_date), MAX(hire_date) 
FROM employees;
"""
cursor.execute(sql)
cursor.fetchall()

[(datetime.date(1985, 1, 1), datetime.date(2000, 1, 28))]

Очікуваний результат:  
    `[(datetime.date(2000, 1, 28), datetime.date(1985, 1, 1))]`

4.Давайте зберемо інформацію про останню ЗП кожного співробіткника в базі. З використанням `con.cursor()` виберіть з таблиці `salaries` для кожного співробітника його номер, останню в базі даних (тобто ту, що відповідає найновішій `to_date`) ЗП, та значення `from_date`, `to_date` для цієї ЗП.

Результати запишіть у pandas DataFrame, який збережіть у змінну `res`. Як взяти з курсора назви колонок - вже наведено. На екран виведіть назви колонок в змінній `res`, розмір цього датафрейму та перші 3 записи в ньому.

УВАГА! Зверніть увагу, що ЗП може оновлюватись протягом одного дня, тобто є випадки, коли апдейт ЗП був в певний день, скажімо 2001-01-01 і потім стався ще один апдейт ЗП цьому співробітнику цього самого дня. Запит має це враховувати.
Також, теоретично ЗП може бути зменшена. Тож, брати дату з максимальною ЗП буде неправильно з точки зору бізнес логіки. Треба саме останнє історичне значення ЗП кожного співробітника.

In [29]:
sql = """
SELECT emp_no, salary, from_date, to_date
FROM salaries AS s1
WHERE to_date = (SELECT MAX(to_date)
                 FROM salaries AS s2
                 WHERE s1.emp_no = s2.emp_no)
ORDER BY emp_no;
"""
cursor.execute(sql)


In [31]:
import pandas as pd
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
res = pd.DataFrame(rows, columns=columns)
res

,emp_no,salary,from_date,to_date
0,10001,88958,2002-06-22,9999-01-01
1,10002,72527,2001-08-02,9999-01-01
2,10003,43311,2001-12-01,9999-01-01
3,10004,74057,2001-11-27,9999-01-01
4,10005,94692,2001-09-09,9999-01-01
...,...,...,...,...
300175,499995,52868,2002-06-01,9999-01-01
300176,499996,69501,2002-05-12,9999-01-01
300177,499997,83441,2001-08-26,9999-01-01
300178,499998,55003,2001-12-25,9999-01-01


In [33]:
field_names = [i[0] for i in cursor.description]

Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1lNvzbAebtkj3C2v-I5AE6HkMH8MTI1kA)

5.В утвореній вибірці в змінній `res` скільки значень `to_date` рівні `9999-01-01`, а скільки - ні? Увага! Для виконання цього завдання важливо розуміти, який тип даних у значень колонки `to_date`. Зʼясуйте це, і потім напишіть фільтр для виводу потрібного результату.

In [43]:
print(res.dtypes)

res['to_date'] = res['to_date'].astype(str)

count_9999 = res[res['to_date'] == '9999-01-01'].shape[0]
count_not_9999 = res[res['to_date'] != '9999-01-01'].shape[0]

print(f"Кількість значень to_date рівних '9999-01-01': {count_9999}")
print(f"Кількість значень to_date, які не рівні '9999-01-01': {count_not_9999}")


emp_no        int64
salary        int64
from_date    object
to_date      object
dtype: object
Кількість значень to_date рівних '9999-01-01': 240124
Кількість значень to_date, які не рівні '9999-01-01': 60056


6.Виведіть візуально розподіл (гістограму) зарплат співробітників, які досі працюють в компанії, використовуючи результат попереднього завдання. Співробітниками, які досі працюють в компанії вважаємо тих, в кого to_date має значення 9999-01-01. Обовʼязково підпишіть графік і вісі.

Ці три завдання - приклад, як нам може бути зручно витягнути дані з бази певним запитом, а потім використати витягнуту вибірку у Python для подальшого аналізу.

Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1j8MHLC1E0revbQLM9B0UJs6tVEgoWhmK)

НЕ ЗАБУВАЄМО ЗАКРИТИ ПІДКЛЮЧЕННЯ ДО БАЗИ :)

In [ ]:
con.close()